In [1]:
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten,MaxPooling2D
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import numpy.random as rng
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import keras

Using TensorFlow backend.


In [2]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, AvgPool2D, BatchNormalization, Dropout, Activation, MaxPool2D
from keras.layers import Activation
from keras.layers import MaxPooling2D
from keras.optimizers import Adam
from keras import regularizers
from face_rec import Face_Embedding
from face_rec import Image_Preprocessing
import face_recognition
import pandas as pd
from keras.utils import to_categorical
import cv2
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib 
from sklearn.model_selection import train_test_split
import os
from itertools import combinations

C:\Users\ultra\Anaconda3\envs\ocr_env\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


참조
- https://datascienceschool.net/view-notebook/1bde49133d7d40c0806e78b70513040b/
- https://jayhey.github.io/deep%20learning/2018/02/06/saimese_network/

# 이미지에서 얼굴만 빼고+회전

In [3]:
import shutil

In [7]:
%%time

folder_dir = './make_images/'
# 각 class 폴더 명
face_class = os.listdir(folder_dir)
ip = Image_Preprocessing()
for label, class_ in enumerate(face_class):
    class_dir = os.path.join(folder_dir, class_)
    write_path = os.path.join(class_dir, class_+'_facebox')
    if os.path.exists(write_path):
        shutil.rmtree(write_path)
    os.mkdir(write_path)
    print("{}...".format(class_))
    for img in os.listdir(class_dir):
        if '.jpg' in img or '.png' in img:
            print('img : {}'.format(img))
            try:
                img_path = os.path.join(class_dir, img)
                file_name = os.path.split(img_path)[-1].split('.')[0]
                img_ = ip.face_preprocessing(img_path, True)
                if img_ != 0:
                    cv2.imwrite(os.path.join(write_path, file_name+'_box.jpg'), img_[0])
            except:
                print('error img : {}'.format(img))
                pass
            
            
           

donald_10...
img : donald_glover_1.jpg
img : donald_glover_10.jpg
img : donald_glover_2.jpg
img : donald_glover_3.jpg
img : donald_glover_4.jpg
img : donald_glover_5.jpg
img : donald_glover_6.jpg
img : donald_glover_7.jpg
img : donald_glover_8.jpg
img : donald_glover_9.jpg
emma_10...
img : hermi_1.jpg
img : hermi_10.jpg
img : hermi_2.jpg
img : hermi_3.jpg
img : hermi_4.jpg
img : hermi_5.jpg
img : hermi_6.jpg
img : hermi_7.jpg
img : hermi_8.jpg
img : hermi_9.jpg
finn_10...
img : finn_1.jpg
img : finn_10.jpg
img : finn_2.jpg
img : finn_3.jpg
img : finn_4.jpg
img : finn_5.jpg
img : finn_6.png
img : finn_7.jpg
img : finn_8.jpg
img : finn_9.jpg
freddie_10...
img : freddie_highmore_1.jpg
img : freddie_highmore_10.jpg
img : freddie_highmore_2.jpg
img : freddie_highmore_3.jpg
img : freddie_highmore_4.jpg
img : freddie_highmore_5.jpg
img : freddie_highmore_6.jpg
img : freddie_highmore_7.jpg
img : freddie_highmore_8.jpg
img : freddie_highmore_9.jpg
gong_10...
img : gong1.jpg
img : gong10.jpg
img

## 얼굴만 뽑아낸 이미지를 다른 폴더로 복사

In [8]:
face_list = []
image_folder = './make_images/'
for c in os.listdir(image_folder):
    c_p = os.path.join(image_folder, c)
    for face_ in os.listdir(c_p):
        if 'facebox' in face_:
            face_list.append(os.path.join(c_p,face_))
face_list

['./make_images/donald_10\\donald_10_facebox',
 './make_images/emma_10\\emma_10_facebox',
 './make_images/finn_10\\finn_10_facebox',
 './make_images/freddie_10\\freddie_10_facebox',
 './make_images/gong_10\\gong_10_facebox',
 './make_images/harry_10\\harry_10_facebox',
 './make_images/hyangi_10\\hyangi_10_facebox',
 './make_images/jaden_10\\jaden_10_facebox',
 './make_images/kim_10\\kim_10_facebox',
 './make_images/maisie_10\\maisie_10_facebox',
 './make_images/matilda_10\\matilda_10_facebox',
 './make_images/natalie_10\\natalie_10_facebox',
 './make_images/park_10\\park_10_facebox',
 './make_images/ron_10\\ron_10_facebox',
 './make_images/sin_10\\sin_10_facebox',
 './make_images/wang_10\\wang_10_facebox']

In [10]:
%%time
path_ = './small_traiging_images3/'
for face_ in face_list:
    mkdir_path = os.path.join(path_, os.path.split(face_)[-1])
    shutil.copytree(face_, mkdir_path)

Wall time: 179 ms


# 이미지 쌍 만들기
- (Anchor, Positive), (Anchor, Negative)
    - [(batch, 200, 200, 3), (batch, 200, 200, 3)]
- siames model에서 사용되는 이미지 쌍들
    - 삼중한 손실을 통해 학습을 진행
    - 학습이 잘 되게 하기위해서는 같은이미지의 거리와 다른이미지의 거리 사이가 크게 나오게해야함
    - 그렇게 학습되게 하려면 d(A, P)와 d(A, N) 사이의 차이가 커야하고
        - A와 N을 유사한 이미지를 씀으로써 좀 더 학습이 잘되게 해야함

In [3]:
%%time
image_folder = './small_traiging_images3/'
class_folder = os.listdir(image_folder)
class_dict = {}
image_size = 128
AP_list = []
AN_list = []
for folder, f in enumerate(class_folder):
    folder_path = os.path.join(image_folder,f)
    class_images = os.listdir(folder_path)
    print("{}...".format(folder))
    for i, images in enumerate(class_images):
        if '.jpg' in images or '.png' in images:
            img_path = os.path.join(folder_path, images)
            
            # Anchor image
            img_A = cv2.resize(cv2.imread(img_path), (image_size,image_size)) 
            for pare_idx in range(i+1,len(class_images)):
                pare_path = os.path.join(folder_path, class_images[pare_idx])
                # Positive image
                img_P = cv2.resize(cv2.imread(img_path), (image_size,image_size)) 
                AP_list.append([img_A, img_P])
    
            for folder_idx in range(folder+1, len(class_folder)):
                pare_path = os.path.join(image_folder, class_folder[folder_idx])
                for negative_img in os.listdir(pare_path):
                    img_N_path = os.path.join(pare_path, negative_img)
                    img_N = cv2.resize(cv2.imread(img_N_path), (image_size,image_size)) 
                    AN_list.append([img_A, img_N])
    print("len(AP_list) : {}".format(len(AP_list)))
    print("len(AN_list) : {}".format(len(AN_list)))
class_dict[0] = AN_list
class_dict[1] = AP_list      

0...
len(AP_list) : 45
len(AN_list) : 1530
1...
len(AP_list) : 90
len(AN_list) : 2960
2...
len(AP_list) : 135
len(AN_list) : 4290
3...
len(AP_list) : 180
len(AN_list) : 5520
4...
len(AP_list) : 225
len(AN_list) : 6650
5...
len(AP_list) : 270
len(AN_list) : 7680
6...
len(AP_list) : 375
len(AN_list) : 9000
7...
len(AP_list) : 420
len(AN_list) : 9780
8...
len(AP_list) : 465
len(AN_list) : 10460
9...
len(AP_list) : 510
len(AN_list) : 11040
10...
len(AP_list) : 555
len(AN_list) : 11520
11...
len(AP_list) : 600
len(AN_list) : 11900
12...
len(AP_list) : 636
len(AN_list) : 12161
13...
len(AP_list) : 672
len(AN_list) : 12341
14...
len(AP_list) : 717
len(AN_list) : 12441
15...
len(AP_list) : 762
len(AN_list) : 12441
Wall time: 25.9 s


# input data set
- [(batch, 128, 128, 3), (batch, 128, 128, 3)]

In [4]:
%%time
image_folder = './training_images/'
class_folder = os.listdir(image_folder)
class_dict = {}
equl_list = []
diff_list = []
for folder, f in enumerate(class_folder):
    folder_path = os.path.join(image_folder,f)
    class_images = os.listdir(folder_path)
    print("{}...".format(folder))
    for i, images in enumerate(class_images):
        if '.jpg' in images or '.png' in images:
            img_path = os.path.join(folder_path, images)
            # 랜덤으로 쌍을 만드는데
            pare_image_idx = np.random.randint(len(class_images))
            # 자기자신을 선택하면 다시 선택
            while(pare_image_idx == i):
                pare_image_idx = np.random.randint(len(class_images))
            pare_path = os.path.join(folder_path, class_images[pare_image_idx])
            img1 = cv2.resize(cv2.imread(img_path), (128,128))
            img2 = cv2.resize(cv2.imread(pare_path), (128,128))
            pare_folder_idx = np.random.randint(len(class_folder))
            while(pare_folder_idx == folder):
                pare_folder_idx = np.random.randint(len(class_folder))
            # 지금 이 폴더 빼고 다른 폴데의 경로를 가지고옴 
            diff_folder_path = os.path.join(image_folder, class_folder[pare_folder_idx])
            # 그 폴더내의 이미지목록을 가지고옴
            diff_iamges = os.listdir(diff_folder_path)
            # 랜덤하게 하나 뽑고
            diff_pare_idx = np.random.randint(len(diff_iamges))
            diff_image_path = os.path.join(diff_folder_path, diff_iamges[diff_pare_idx])
            img3 = cv2.resize(cv2.imread(diff_image_path), (128,128))
            # 두 이미지를 rgb채널뒤에 얹어서 더함
            # (96,96,6)의 shape으로 만듬
#             equl_list.append(np.concatenate((img1, img2), axis = 2))
#             diff_list.append(np.concatenate((img1, img3), axis = 2))
            equl_list.append([img1, img2])
            diff_list.append([img1, img3])
#             print("img1 : {} img2 : {} img3 : {}".format(str.split(images, '.')[0],
#                                                          str.split(class_images[pare_image_idx], '.')[0],
#                                                          str.split(diff_iamges[diff_pare_idx], '.')[0]))
            
class_dict[0] = diff_list
class_dict[1] = equl_list

0...
1...
2...
3...
4...
5...
Wall time: 43.2 s


In [37]:
# %%time
# from multiprocessing import Pool
# import np_preprocessing

# np_shape = 200
# np_1 = np.zeros((1,np_shape,np_shape,3))
# np_2 = np.zeros((1,np_shape,np_shape,3))
# if __name__ ==  '__main__': 
#     with Pool(8) as p:
#         np_1, np_2 = p.starmap(np_preprocessing.np_append,[(np_1, np_2, class_dict,np_shape)])

In [38]:
# from joblib import Parallel, delayed
# %%time
# def np_append(np_1,np_2, class_dict,np_shape=200):
#     np_1 = np.zeros((1,np_shape,np_shape,3))
#     np_2 = np.zeros((1,np_shape,np_shape,3))
#     for k in class_dict.keys():
#         for i, arr_list in enumerate(class_dict[k]):
#             if i % 20 == 0:
#                 print('{} : {} / {} ...'.format(k, i, len(class_dict[k])))
#             for i_, np_arr in enumerate(arr_list):
#                 img_ = np_arr.reshape((-1,np_shape,np_shape,3))
#                 if i_ == 0:
#                     np_1 = np.append(np_1, img_, axis = 0)
#                 else:
#                     np_2 = np.append(np_2, img_, axis = 0)
#     return np_1, np_2

# # 분산처리 작업
# with Parallel(n_jobs=10, verbose=1, prefer ='threads') as ex:
#     # Image column을 분리해서 list로 만들고 np.array로 바꿈
#     np_1_test, np_2_test = ex(delayed(np_append)(np_1, np_2, class_dict, 200))


In [5]:
%%time

np_1 = np.zeros((1,np_shape,np_shape,3))
np_2 = np.zeros((1,np_shape,np_shape,3))
for k in class_dict.keys():
    for i, arr_list in enumerate(class_dict[k]):
        if i % 20 == 0:
            print('{} : {} / {} ...'.format(k, i, len(class_dict[k])))
        for i_, np_arr in enumerate(arr_list):
            img_ = np_arr.reshape((-1,np_shape,np_shape,3))
            if i_ == 0:
                np_1 = np.append(np_1, img_, axis = 0)
            else:
                np_2 = np.append(np_2, img_, axis = 0)

0 : 0 / 1750 ...
0 : 20 / 1750 ...
0 : 40 / 1750 ...
0 : 60 / 1750 ...
0 : 80 / 1750 ...
0 : 100 / 1750 ...
0 : 120 / 1750 ...
0 : 140 / 1750 ...
0 : 160 / 1750 ...
0 : 180 / 1750 ...
0 : 200 / 1750 ...
0 : 220 / 1750 ...
0 : 240 / 1750 ...


KeyboardInterrupt: 

# 자기 이외에 다른 이미지 다 잇기

기존의 dict 형태  
class_dict[0] = [np.array, np.array, ...] 각 np.array는 (128, 128, 6)  
아래의 dict 형태  
class_dict[0] = [[np.array, np.array], [np.array, np.array], ...]  각 np.array는 (128,128,3)  

In [143]:
len(class_dict[1][0])

2

# np.append
- [(batch, 128, 128, 3), (batch, 128, 128, 3)] 형태의 numpy array만들기 위해 
- [ [np.arr, np.arr], [np.arr, np.arr]] 을 뜯어서 아래와 같이 append

In [14]:
np_1 = np_1[1:]
np_2 = np_2[1:]

In [88]:
print("np_1 shape : {} np_2 shape : {}".format(np_1.shape, np_2.shape))

np_1 shape : (2080, 200, 200, 3) np_2 shape : (2080, 200, 200, 3)


In [16]:
y_data = []
for k in class_dict.keys():
    for _ in class_dict[k]:
        y_data.append(k)
y_data = np.array(y_data)
print(y_data.shape)
print(y_data[:3])

(2080,)
[0 0 0]


In [ ]:
np

In [44]:
y_data

array([0, 0, 0, ..., 1, 1, 1])

In [45]:
y_data_one_hot = to_categorical(y_data)

# np_save

In [19]:
# 200 np save
np.savez('./200_cnn_np', x_data1 = np_1, x_data2 = np_2, y_data = y_data)

In [172]:
np.savez('./200_cnn_np2', x_data1 = np_1, x_data2 = np_2, y_data = y_data)

- CNN layer를 2개만듬
- 각 layer의 마지막단계에서 나오게되는 특징값들을 펼치고
- distance metrics를 구함


In [ ]:
른 레이어(Tensor) 객체를 입력으로 호출하면 그 레이어를 입력으로 가지는 복합 레이어 객체가 된다.
Model 클래스 객체는 Input 레이어와 그 Input 레이어에 연결된 다른 레이어를 출력으로 주어 생성.
Model 클래스 객체도 다른 레이어(텐서)를 입력으로 호출하면 그 레이어를 입력으로 가지는 복합 레이어가 된다.

# np_load

In [3]:
data = np.load('./200_cnn_np.npz')

In [3]:
data = np.load('./200_cnn_np2.npz')

In [4]:
np_1 = data['x_data1']
np_2 = data['x_data2']
y_data = data['y_data']
print("np_1.shape : {} np_2.shape: {}, y_data.shape : {}".format(np_1.shape, np_2.shape, y_data.shape))

np_1.shape : (660, 200, 200, 3) np_2.shape: (660, 200, 200, 3), y_data.shape : (660,)


In [142]:
scaler = MinMaxScaler()
np_1_norm = scaler.fit_transform(np_1.reshape(-1, 200*200*3))
np_2_norm = scaler.fit_transform(np_2.reshape(-1, 200*200*3))
np_1_norm = np_1_norm.reshape(-1,200,200,3)
np_2_norm = np_2_norm.reshape(-1,200,200,3)

In [5]:
x1_train, x1_val, x2_train, x2_val, y_train, y_val = train_test_split(np_1, np_2, y_data, test_size = 0.2, random_state = 42)

In [143]:
x1_train, x1_val, x2_train, x2_val, y_train, y_val = train_test_split(np_1_norm, np_2_norm, y_data, test_size = 0.2, random_state = 42)

In [ ]:
scaler = MinMaxScaler()
# split
x_train, x_val, y_train, y_val = train_test_split(x_data,y_data,test_size = 0.2, random_state = 42)
print("x_train.shape :{}\nx_val.shape : {}".format(x_train.shape, x_val.shape))
print("y_train.shape :{}\ny_val.shape : {}".format(y_train.shape, y_val.shape))

# 데이터 정규화
x_train_norm = x_train[:,:,:,:]
x_val_norm = x_val[:,:,:,:]
x_train_norm = x_train_norm.reshape([-1, img_size*img_size*img_shape])
x_val_norm = x_val_norm.reshape([-1,img_size*img_size*img_shape])
print("x_train_norm.shape : {}".format(x_train_norm.shape))
print("x_val_norm,.shape : {}".format(x_val_norm.shape))
x_train_norm = scaler.fit_transform(x_train_norm)
x_val_norm = scaler.fit_transform(x_val_norm)
print("x_train_norm.mean : {}".format(x_train_norm.mean()))
print("x_train_norm.std : {}".format(x_train_norm.std()))
# reshape
x_train_norm = x_train_norm.reshape([-1,img_size,img_size,img_shape])
x_val_norm = x_val_norm.reshape([-1,img_size,img_size,img_shape])
print("x_train_norm.shape : {}".format(x_train_norm.shape))
print("x_val_norm,.shape : {}".format(x_val_norm.shape))
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
print("y_train.shape {}, y_val.shape : {}".format(y_train.shape, y_val.shape))

In [6]:
print("x1_train.shape : {}\nx1_val.shape : {}\nx2_train.shape : {}\nx2_val.shape : {} ".format(x1_train.shape, x1_val.shape, x2_train.shape, x2_val.shape))
print("y_train.shape : {}\ny_val.shape : {}".format(y_train.shape, y_val.shape))

x1_train.shape : (528, 200, 200, 3)
x1_val.shape : (132, 200, 200, 3)
x2_train.shape : (528, 200, 200, 3)
x2_val.shape : (132, 200, 200, 3) 
y_train.shape : (528,)
y_val.shape : (132,)


## 원본
- weight, bias init => he_normal
- 링크 : https://github.com/sorenbouma/keras-oneshot/blob/master/SiameseNet.ipynb

In [13]:
%matplotlib inline
# def W_init(shape,name=None):
#     """Initialize weights as in paper"""
#     values = rng.normal(loc=0,scale=1e-2,size=shape)
#     return K.variable(values,name=name)
# #//TODO: figure out how to initialize layer biases in keras.
# def b_init(shape,name=None):
#     """Initialize bias as in paper"""
#     values=rng.normal(loc=0.5,scale=1e-2,size=shape)
#     return K.variable(values,name=name)

input_shape = (128, 128, 3)
left_input = Input(input_shape)
right_input = Input(input_shape)
#build convnet to use in each siamese 'leg'
convnet = Sequential()
#convnet.add(Conv2D(48,(10,10),activation='relu',input_shape=input_shape,kernel_initializer=W_init,kernel_regularizer=l2(2e-4)))
convnet.add(Conv2D(48,(10,10),activation = 'relu', input_shape=input_shape,kernel_initializer='he_normal',kernel_regularizer=l2(2e-4)))

convnet.add(MaxPooling2D())
#convnet.add(Conv2D(36,(7,7),activation='relu',kernel_regularizer=l2(2e-4),kernel_initializer=W_init,bias_initializer=b_init))
convnet.add(Conv2D(36,(7,7),activation = 'relu',kernel_regularizer=l2(2e-4),kernel_initializer='he_normal',bias_initializer='he_normal'))

convnet.add(MaxPooling2D())
#convnet.add(Conv2D(96,(4,4),activation='relu',kernel_initializer=W_init,kernel_regularizer=l2(2e-4),bias_initializer=b_init))
convnet.add(Conv2D(96,(4,4),activation = 'relu', kernel_initializer='he_normal',kernel_regularizer=l2(2e-4),bias_initializer='he_normal'))

convnet.add(MaxPooling2D())
#convnet.add(Conv2D(64,(3,3),activation='relu',kernel_initializer=W_init,kernel_regularizer=l2(2e-4),bias_initializer=b_init))
#convnet.add(Conv2D(48,(3,3),activation='relu',kernel_initializer='he_normal',kernel_regularizer=l2(2e-4),bias_initializer='he_normal'))

#convnet.add(Dropout(0.7))
convnet.add(Flatten())
convnet.add(Dense(512,activation="sigmoid",kernel_regularizer=l2(1e-3),kernel_initializer='he_normal',bias_initializer='he_normal'))
#convnet.add(Dense(1024,activation="sigmoid",kernel_regularizer=l2(1e-3),kernel_initializer='he_normal',bias_initializer='he_normal'))
#convnet.add(Dense(128,activation="sigmoid",kernel_regularizer=l2(1e-3),kernel_initializer='he_normal',bias_initializer='he_normal'))

#call the convnet Sequential model on each of the input tensors so params will be shared
encoded_l = convnet(left_input)
encoded_r = convnet(right_input)
#layer to merge two encoded inputs with the l1 distance between them
# 임의의 표현식을 Layer객체로 래핑(여기선 CNN거치고 나온 각 이미지가 서로의 차를 송출하는 layer를 만듬)
# keras는 tensorflow의텐서 곱셈이라든지 같은 저수준 연산을 처리하지 않음
# 하지만 keras의 backend를 통해 tensorflow의 연산을 수행할 수 있음(여기선 텐서간 차를 구하고 절대값)
L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
#call this layer on list of two input tensors.
L1_distance = L1_layer([encoded_l, encoded_r])
#add_dense = Dense(84, activation='relu',kernel_regularizer=l2(1e-3),kernel_initializer='he_normal',bias_initializer='he_normal')(L1_distance)
prediction = Dense(1,activation='sigmoid',bias_initializer='he_normal')(L1_distance)
siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)

optimizer = Adam(0.00006)
#optimzer = keras.optimizers.SGD(lr=0.0001)
#//TODO: get layerwise learning rates and momentum annealing scheme described in paperworking
siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer, metrics=['accuracy'])


In [14]:
siamese_net.summary(), siamese_net.input, siamese_net.output

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
sequential_5 (Sequential)       (None, 512)          6102452     input_9[0][0]                    
                                                                 input_10[0][0]                   
__________________________________________________________________________________________________
lambda_5 (Lambda)               (None, 512)          0           sequential_5[1][0]               
          

(None,
 [<tf.Tensor 'input_9:0' shape=(?, 128, 128, 3) dtype=float32>,
  <tf.Tensor 'input_10:0' shape=(?, 128, 128, 3) dtype=float32>],
 <tf.Tensor 'dense_10/Sigmoid:0' shape=(?, 1) dtype=float32>)

In [15]:
input_data = [x1_train, x2_train]
val_data = [[x1_val, x2_val], y_val]

In [16]:
len(input_data[0])

1219

In [ ]:
log = siamese_net.fit(input_data, y_train, epochs = 30, batch_size = 2, validation_data= val_data)

Train on 1219 samples, validate on 305 samples
Epoch 1/30
 850/1219 [===================>..........] - ETA: 39s - loss: 1.6071 - acc: 0.5094

In [ ]:
def show_results(*logs):
    trn_loss, val_loss, trn_acc, val_acc = [],[],[],[]
    
    for log in logs:
        trn_loss += log.history['loss']
        val_loss += log.history['val_loss']
        
    fig, ax = plt.subplots(figsize = (8,4))
    # loss value
    ax.plot(trn_loss, label = 'train')
    # accuracy
    ax.plot(val_loss, label = 'validation')
    ax.set_xlabel('epoch')
    ax.set_ylabel('loss')
    ax.legend()

In [ ]:
show_results(log)

# testing


In [21]:
def resize_img2(img_path):
    img_size = 200
    return cv2.resize(cv2.imread(img_path), (img_size,img_size))

In [20]:
ori_path = './small_traiging_images2/emma_10_extend/hermi_1_box.jpg'
cmp_path1 = './small_traiging_images2/emma_10_extend/hermi_3_box.jpg'
cmp_path2 = './small_traiging_images2/finn_10_extend/finn_2_box.jpg'
cmp_path3 = './small_traiging_images2/finn_10_extend/finn_7_box.jpg'
cmp_path4 = './small_traiging_images2/harry_10_extend/harry_6_box.jpg'

In [34]:
ori_path = './small_traiging_images2/emma_10_extend/hermi_1_box.jpg'
cmp_path1 = './small_traiging_images2/emma_10_extend/hermi_2_box.jpg'
cmp_path2 = './small_traiging_images2/emma_10_extend/hermi_3_box.jpg'
cmp_path3 = './small_traiging_images2/emma_10_extend/hermi_4_box.jpg'
cmp_path4 = './small_traiging_images2/emma_10_extend/hermi_5_box.jpg'

In [22]:
original_img = resize_img2(ori_path)
compared_img1 = resize_img2(cmp_path1)
compared_img2 = resize_img2(cmp_path2)
compared_img3 = resize_img2(cmp_path3)
compared_img4 = resize_img2(cmp_path4)

In [23]:
img_shape = 200
original_img = original_img.reshape([-1,img_shape,img_shape,3])
compared_img1 = compared_img1.reshape([-1,img_shape,img_shape,3])
compared_img2 = compared_img2.reshape([-1,img_shape,img_shape,3])
compared_img3 = compared_img3.reshape([-1,img_shape,img_shape,3])
compared_img4 = compared_img4.reshape([-1,img_shape,img_shape,3])

In [37]:
original_img.shape

(1, 200, 200, 3)

In [38]:
siamese_net.predict([original_img, compared_img1])

array([[0.8266766]], dtype=float32)

In [42]:
siamese_net.predict([original_img, compared_img2])

array([[0.6332958]], dtype=float32)

In [43]:
siamese_net.predict([original_img, compared_img3])

array([[0.6332958]], dtype=float32)

In [45]:
siamese_net.predict([original_img, compared_img4])

array([[0.2414935]], dtype=float32)

# 새로운 쌍 만들기..(같은 사진과 다른사진의 비율을 같게)

In [3]:
%%time
image_folder = './small_traiging_images3/'
class_folder = os.listdir(image_folder)
image_shape = 128
class_dict = {}
AP_list = []
AN_list = []
for folder, f in enumerate(class_folder):
    folder_path = os.path.join(image_folder,f)
    class_images = os.listdir(folder_path)
    print("{}...".format(folder))
    for i, images in enumerate(class_images):
        if '.jpg' in images or '.png' in images:
            img_path = os.path.join(folder_path, images)
            
            # Anchor image
            img_A = cv2.resize(cv2.imread(img_path), (image_shape,image_shape)) 
            for pare_idx in range(i+1,len(class_images)):
                pare_path = os.path.join(folder_path, class_images[pare_idx])
                # Positive image
                img_P = cv2.resize(cv2.imread(pare_path), (image_shape,image_shape)) 
                AP_list.append([img_A, img_P])
                # Negative image
                pare_folder_idx = np.random.randint(len(class_folder))
                while(pare_folder_idx == folder):
                    pare_folder_idx = np.random.randint(len(class_folder))
                # 지금 이 폴더 빼고 다른 폴데의 경로를 가지고옴 
                diff_folder_path = os.path.join(image_folder, class_folder[pare_folder_idx])
                # 그 폴더내의 이미지목록을 가지고옴
                diff_iamges = os.listdir(diff_folder_path)
                # 랜덤하게 하나 뽑고
                diff_pare_idx = np.random.randint(len(diff_iamges))
                diff_image_path = os.path.join(diff_folder_path, diff_iamges[diff_pare_idx])
                img_N = cv2.resize(cv2.imread(diff_image_path), (image_shape,image_shape))
                AN_list.append([img_A, img_N])
            
                    
    print("len(AP_list) : {}".format(len(AP_list)))
    print("len(AN_list) : {}".format(len(AN_list)))
class_dict[0] = AN_list
class_dict[1] = AP_list      

0...
len(AP_list) : 45
len(AN_list) : 45
1...
len(AP_list) : 90
len(AN_list) : 90
2...
len(AP_list) : 135
len(AN_list) : 135
3...
len(AP_list) : 180
len(AN_list) : 180
4...
len(AP_list) : 225
len(AN_list) : 225
5...
len(AP_list) : 270
len(AN_list) : 270
6...
len(AP_list) : 375
len(AN_list) : 375
7...
len(AP_list) : 420
len(AN_list) : 420
8...
len(AP_list) : 465
len(AN_list) : 465
9...
len(AP_list) : 510
len(AN_list) : 510
10...
len(AP_list) : 555
len(AN_list) : 555
11...
len(AP_list) : 600
len(AN_list) : 600
12...
len(AP_list) : 636
len(AN_list) : 636
13...
len(AP_list) : 672
len(AN_list) : 672
14...
len(AP_list) : 717
len(AN_list) : 717
15...
len(AP_list) : 762
len(AN_list) : 762
Wall time: 4.46 s


In [34]:
%%time
np_shape = 128
np_1 = np.zeros((1,np_shape,np_shape,3))
np_2 = np.zeros((1,np_shape,np_shape,3))
for k in class_dict.keys():
    for i, arr_list in enumerate(class_dict[k]):
        if i % 50 == 0:
            print('{} : {} / {} ...'.format(k, i, len(class_dict[k])))
        for i_, np_arr in enumerate(arr_list):
            img_ = np_arr.reshape((-1,np_shape,np_shape,3))
            if i_ == 0:
                np_1 = np.append(np_1, img_, axis = 0)
            else:
                np_2 = np.append(np_2, img_, axis = 0)

0 : 0 / 762 ...
0 : 50 / 762 ...
0 : 100 / 762 ...
0 : 150 / 762 ...
0 : 200 / 762 ...
0 : 250 / 762 ...
0 : 300 / 762 ...
0 : 350 / 762 ...
0 : 400 / 762 ...
0 : 450 / 762 ...
0 : 500 / 762 ...
0 : 550 / 762 ...
0 : 600 / 762 ...
0 : 650 / 762 ...
0 : 700 / 762 ...
0 : 750 / 762 ...
1 : 0 / 762 ...
1 : 50 / 762 ...
1 : 100 / 762 ...
1 : 150 / 762 ...
1 : 200 / 762 ...
1 : 250 / 762 ...
1 : 300 / 762 ...
1 : 350 / 762 ...
1 : 400 / 762 ...
1 : 450 / 762 ...
1 : 500 / 762 ...
1 : 550 / 762 ...
1 : 600 / 762 ...
1 : 650 / 762 ...
1 : 700 / 762 ...
1 : 750 / 762 ...
Wall time: 6min 25s


In [43]:
np_1 = np_1[1:]
np_2 = np_2[1:]

In [9]:
y_data = []
for k in class_dict.keys():
    for _ in class_dict[k]:
        y_data.append(k)
y_data = np.array(y_data)
print(y_data.shape)
print(y_data[:3])

(1524,)
[0 0 0]


# np save

In [10]:
# 200 np save
np.savez('./128_real_cnn_np', x_data1 = np_1, x_data2 = np_2, y_data = y_data)

In [24]:
# 200 np save
np.savez('./200_real_cnn_np', x_data1 = np_1, x_data2 = np_2, y_data = y_data)

# np load

In [5]:
data = np.load('./200_real_cnn_np.npz')
np_1 = data['x_data1']
np_2 = data['x_data2']
y_data = data['y_data']
print("np_1.shape : {} np_2.shape: {}, y_data.shape : {}".format(np_1.shape, np_2.shape, y_data.shape))

np_1.shape : (1524, 200, 200, 3) np_2.shape: (1524, 200, 200, 3), y_data.shape : (1524,)


In [11]:
data = np.load('./128_real_cnn_np.npz')
np_1 = data['x_data1']
np_2 = data['x_data2']
y_data = data['y_data']
print("np_1.shape : {} np_2.shape: {}, y_data.shape : {}".format(np_1.shape, np_2.shape, y_data.shape))

np_1.shape : (1524, 128, 128, 3) np_2.shape: (1524, 128, 128, 3), y_data.shape : (1524,)


In [12]:
x1_train, x1_val, x2_train, x2_val, y_train, y_val = train_test_split(np_1, np_2, y_data, test_size = 0.2, random_state = 42)
print("x1_train.shape : {}\nx1_val.shape : {}\nx2_train.shape : {}\nx2_val.shape : {} ".format(x1_train.shape, x1_val.shape, x2_train.shape, x2_val.shape))
print("y_train.shape : {}\ny_val.shape : {}".format(y_train.shape, y_val.shape))

x1_train.shape : (1219, 128, 128, 3)
x1_val.shape : (305, 128, 128, 3)
x2_train.shape : (1219, 128, 128, 3)
x2_val.shape : (305, 128, 128, 3) 
y_train.shape : (1219,)
y_val.shape : (305,)


# model save/load

In [18]:
from keras.models import load_model

# Creates a HDF5 file 'my_model.h5'
siamese_net.save('siamese_net_200_test.h5')

In [31]:
from keras.models import load_model
# Returns a compiled model identical to the previous one
model = load_model('./siamese_net_200_test.h5')

img_size = 200
origianl_img = cv2.resize(cv2.imread(ori_path), (img_size,img_size))

model.predict(([original_img, compared_img1]))

array([[0.7392402]], dtype=float32)

In [52]:
resize_reshape_img(original_img_path, 200).shape

(1, 200, 200, 3)

# 가장 유사한 이미지 이름과 acc 리턴

In [69]:
%%time
# image를 resize해주고 reshape.. 인자로 size를 줘야함
def resize_reshape_img(img_path, img_size):
    img = cv2.resize(cv2.imread(img_path),(img_size,img_size))
    return img.reshape([-1, img_size, img_size, 3])

img_path = './small_traiging_images2/'
original_img_path = './small_traiging_images2/emma_10_extend/hermi_1_box.jpg'
# original img <- 들어오는 이미지 shape과 size를 바꿔줌
original_img = resize_reshape_img(original_img_path, 200)

acc_list = []
file_list = []
# 전체경로에서 각 이미지 폴더에 접근
for folder in os.listdir(img_path):
    folder_path = os.path.join(img_path, folder)
    # 각 이미지 폴더에서 폴더내 이미지 접근
    for file in os.listdir(folder_path):
        print("{} predict...".format(file))
        file_path = os.path.join(folder_path, file)
        # db폴더내의 각 이미지에 대해 resize , reshape
        compare_file = resize_reshape_img(file_path, 200)
        file_list.append(str.split(file, '.')[0])
        acc_list.append(model.predict([original_img, compare_file])[0])
acc_np = np.array(acc_list)        
max_idx = np.argmax(acc_np)
print('file name : {}, acc : {}'.format(file_list[max_idx], acc_np[max_idx]))

hermi_10_box.jpg predict...
hermi_1_box.jpg predict...
hermi_2_box.jpg predict...
hermi_3_box.jpg predict...
hermi_4_box.jpg predict...
hermi_5_box.jpg predict...
hermi_6_box.jpg predict...
hermi_7_box.jpg predict...
hermi_8_box.jpg predict...
hermi_9_box.jpg predict...
finn_10_box.jpg predict...
finn_1_box.jpg predict...
finn_2_box.jpg predict...
finn_3_box.jpg predict...
finn_4_box.jpg predict...
finn_5_box.jpg predict...
finn_6_box.png predict...
finn_7_box.jpg predict...
finn_8_box.jpg predict...
finn_9_box.jpg predict...
harry_10_box.jpg predict...
harry_1_box.jpg predict...
harry_2_box.jpg predict...
harry_3_box.jpg predict...
harry_4_box.jpg predict...
harry_5_box.jpg predict...
harry_6_box.jpg predict...
harry_7_box.jpg predict...
harry_8_box.jpg predict...
harry_9_box.jpg predict...
file name : hermi_10_box, acc : [0.7392402]
Wall time: 1.57 s


In [1]:
import keras
print(keras.__version__)

Using TensorFlow backend.


2.2.4
